### Import

In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, MultiLabelBinarizer
from sklearn.ensemble import RandomForestClassifier

### Data Load

In [2]:
train = pd.read_csv('selected_features_cleaned.csv').drop(columns=['ID'])
test = pd.read_csv('test.csv').drop(columns=['ID'])

missing_cols = set(test.columns) - set(train.columns)
test = test.drop(columns=missing_cols)

In [3]:
X = train.drop('임신 성공 여부', axis=1)
y = train['임신 성공 여부']

### Data Pre-Processing

In [4]:
# 범주형 변수 찾기
categorical_features = X.select_dtypes(include=['object', 'category']).columns

# OneHotEncoder 설정
ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

# 원-핫 인코딩 수행
X_encoded = pd.DataFrame(ohe.fit_transform(X[categorical_features]), columns=ohe.get_feature_names_out(categorical_features))

# 원-핫 인코딩된 데이터프레임과 수치형 변수 결합
X = X.drop(columns=categorical_features).reset_index(drop=True)
X = pd.concat([X, X_encoded], axis=1)

# 테스트 데이터에도 동일한 변환 적용
test_encoded = pd.DataFrame(ohe.transform(test[categorical_features]), columns=ohe.get_feature_names_out(categorical_features))
test = test.drop(columns=categorical_features).reset_index(drop=True)
test = pd.concat([test, test_encoded], axis=1)

# 변환된 데이터 확인
print(X.head())
print(test.head())

   배란 자극 여부  단일 배아 이식 여부  착상 전 유전 진단 사용 여부  남성 주 불임 원인  남성 부 불임 원인  \
0         1          0.0               0.0           0           0   
1         1          0.0               0.0           0           0   
2         1          0.0               0.0           0           0   
3         1          0.0               0.0           0           0   
4         1          0.0               0.0           0           0   

   여성 주 불임 원인  여성 부 불임 원인  부부 주 불임 원인  부부 부 불임 원인  불명확 불임 원인  ...  \
0           0           0           0           0          0  ...   
1           0           0           0           0          1  ...   
2           0           0           0           0          1  ...   
3           0           0           0           0          0  ...   
4           0           0           0           0          1  ...   

   정자 출처_기증 제공  정자 출처_배우자 제공  난자 기증자 나이_알 수 없음  정자 기증자 나이_만20세 이하  \
0          0.0           1.0               1.0                0.0   
1          0.0           1

### Train

In [5]:
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_encoded, y)

RandomForestClassifier(random_state=42)

In [6]:
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

# 80% 훈련, 20% 검증 데이터로 분리
X_train, X_val, y_train, y_val = train_test_split(X_encoded, y, test_size=0.2, random_state=42, stratify=y)

# 모델 학습
#model = RandomForestClassifier(n_estimators=100, random_state=42)
#model = RandomForestClassifier(n_estimators=100, max_depth=10, min_samples_split=10, min_samples_leaf=5, random_state=42)
model = RandomForestClassifier(n_estimators=300, max_depth=10, min_samples_split=10, min_samples_leaf=5, random_state=42)
model.fit(X_train, y_train)

# 검증 데이터에 대한 성능 평가
y_val_pred = model.predict(X_val)
y_val_proba = model.predict_proba(X_val)[:, 1]

accuracy = accuracy_score(y_val, y_val_pred)
roc_auc = roc_auc_score(y_val, y_val_proba)

print(f"Validation Accuracy: {accuracy:.4f}")
print(f"Validation ROC-AUC Score: {roc_auc:.4f}")

# 5-Fold Cross Validation
cv_scores = cross_val_score(model, X_encoded, y, cv=5, scoring="roc_auc")

print(f"Cross-Validation ROC-AUC Scores: {cv_scores}")
print(f"Mean ROC-AUC Score: {cv_scores.mean():.4f}")

Validation Accuracy: 0.7419
Validation ROC-AUC Score: 0.6332
Cross-Validation ROC-AUC Scores: [0.62875187 0.62734417 0.62681483 0.6484609  0.63099544]
Mean ROC-AUC Score: 0.6325


### Predict

In [ ]:
pred_proba = model.predict_proba(test_encoded)[:, 1]

### Submission

In [ ]:
sample_submission = pd.read_csv('sample_submission.csv')
sample_submission['probability'] = pred_proba

In [ ]:
sample_submission.to_csv('./encoding_test_submit2.csv', index=False)